In [2]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.5.0
cuda


In [3]:
(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

In [4]:
[idx_to_char[ id] for id in corpus_indices[0:10]]

['想', '要', '有', '直', '升', '机', ' ', '想', '要', '和']

## 6.4.1 one-hot向量

In [5]:
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1) # scatter_(dim, index, src)
    return res
    
x = torch.tensor([0, 2])
one_hot(x, vocab_size)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.]])

In [6]:
x = torch.tensor([0, 2])
x = x.long()
x.view(-1, 1)

tensor([[0],
        [2]])

In [7]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

X = torch.arange(10).view(2, 5)
inputs = to_onehot(X, vocab_size)
print(len(inputs), inputs[0].shape)
print(inputs[1][:,0:10])
print(X)

5 torch.Size([2, 1027])
tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])
tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])


## 6.4.2 初始化模型参数

In [8]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
print('will use', device)

def get_params():
    def _one(shape):
        ts = torch.tensor(np.random.normal(0, 0.01, size=shape), device=device, dtype=torch.float32)
        return torch.nn.Parameter(ts, requires_grad=True)

    # 隐藏层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = torch.nn.Parameter(torch.zeros(num_hiddens, device=device, requires_grad=True))
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = torch.nn.Parameter(torch.zeros(num_outputs, device=device, requires_grad=True))
    return nn.ParameterList([W_xh, W_hh, b_h, W_hq, b_q])

will use cuda


## 6.4.3 定义模型

In [9]:
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), device=device), )

In [10]:
def rnn(inputs, state, params):
    # inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = torch.tanh(torch.matmul(X, W_xh) + torch.matmul(H, W_hh) + b_h)
        Y = torch.matmul(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [11]:
state = init_rnn_state(X.shape[0], num_hiddens, device)
inputs = to_onehot(X.to(device), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
print(len(outputs), outputs[0].shape, state_new[0].shape)

5 torch.Size([2, 1027]) torch.Size([2, 256])


## 6.4.4 定义预测函数

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, device, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, device)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot(torch.tensor([[output[-1]]], device=device), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [14]:
predict_rnn('天晴', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            device, idx_to_char, char_to_idx)

'天晴装欣骷躲将片头铅酋不'

## 6.4.5 裁剪梯度

In [15]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def grad_clipping(params, theta, device):
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## 6.4.6 困惑度
## 6.4.7 定义模型训练函数

In [16]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, device, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = nn.CrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, device)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, device)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, device)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach_()
            
            inputs = to_onehot(X, vocab_size)
            # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
            (outputs, state) = rnn(inputs, state, params)
            # 拼接之后形状为(num_steps * batch_size, vocab_size)
            outputs = torch.cat(outputs, dim=0)
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            # 使用交叉熵损失计算平均分类误差
            l = loss(outputs, y.long())
            
            # 梯度清0
            if params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            grad_clipping(params, clipping_theta, device)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, device, idx_to_char, char_to_idx))

## 6.4.8 训练模型并创作歌词

In [19]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 2500, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [20]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 71.266550, time 0.09 sec
 - 分开 我不要再想 我不能再想 你不能 一颗两颗三颗四步  我想能你 你有了我 我有我不 你是我 别子我有
 - 不分开 快颗用 一颗两颗三颗四步  我想能你 你有我有 你是我 别子我有三你的让我 甩的可我 我有你你 你
epoch 100, perplexity 10.346547, time 0.09 sec
 - 分开 一颗两步三步四颗 连成线背著背我妈妈 难说的手不着 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍
 - 不分开吗 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 
epoch 150, perplexity 2.832930, time 0.09 sec
 - 分开 一直会停 语我用练 习成风中 你在没在 你一定梦 一句不起 你不懂 连一句珍重 一切苦斗飞 有人什
 - 不分开吗 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200, perplexity 1.578092, time 0.09 sec
 - 分开 有蟑去人教 几钩都苦落 三为什么落 不爽都 岩给堂 装满都明信片的铁盒里藏著一片玫瑰花瓣 对金葛爬
 - 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 一子我想见的我 一思风枪 目我将
epoch 250, perplexity 1.310064, time 0.09 sec
 - 分开 一天心停留 它辛它苦抽 一朵一朵因你而香 试图让夕阳飞翔 带领你我环绕大自然 迎著风 开始共渡每一
 - 不分开吗 我叫你爸 你打我妈 这样对吗干嘛这样 何必让酒牵口子走 瞎 说在灌口 我该了你不个 但不会掩 如
epoch 300, perplexity 1.218886, time 0.09 sec
 - 分开 有直在人蛛  辛上依旧不会 老天不起格永远 但来杂草的泥土 不会莹明武 会学坊 小弄堂 我打就这样
 - 不分开简 然后将过去 慢慢温习 让我爱上你 那场悲剧 是你完美演出的一场戏 宁愿心碎哭泣 再狠狠忘记 你爱
epoch 350, perplexity 1.183997, time 0.08 sec

In [21]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, device, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 63.156232, time 0.08 sec
 - 分开 我想要这不 我不要你想 我不要这想 我不要这想 我不要你想 我不要你想 我不要这想 我不要这想 我
 - 不分开 我想要你想 我不要的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的
epoch 100, perplexity 7.311214, time 0.09 sec
 - 分开 一颗了 娘颗我 我有就这宣牵 你你 却又再考倒我 说散 你想很久了吧? 我不要的黑色在西 不要我说
 - 不分开步妈会 一直线 分手的颗我语狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的
epoch 150, perplexity 2.062818, time 0.09 sec
 - 分开 让候我 谁有开枪我 开不和的手有 让我到回忆了 瞎想都 我想你 一九就的手说 想真 却又再考倒我 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 200, perplexity 1.309920, time 0.09 sec
 - 分开 你候我 谁透神枪的我不想我一泪拆 牵托 一场盯酒 戒指伦 干沉我 印地安的传说 还真在 瞎透了 什
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了双截棍 风生哈兮 是使用双截棍 哼哼哈兮 
epoch 250, perplexity 1.146958, time 0.08 sec
 - 分开 你候啊 你怎么打我手 你说啊 是不是你不想活 说你怎么面对我 甩开球我满腔的怒火 我想揍你已经很久
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 300, perplexity 1.115363, time 0.08 sec
 - 分开 你候 这不着 后什么 我想大的手二 像 在你忆直棒着 在这样的生活 我爱你 你爱上 我想开声宣布 
 - 不分开觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生
epoch 350, perplexity 1.121679, time 0.08 sec
